In [ ]:
""" Simple Auto Encoder """

In [7]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [17]:
# Parameters
learning_rate = 0.01
training_epochs = 5
batch_size      = 100
display_step = 1
plot_step       = 10

# Network Parameters
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 256 # 2nd layer num features
n_input    = 784 # MNIST data input (img shape: 28*28)
n_output   = 784 # 

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_output])
dropout_keep_prob = tf.placeholder("float")

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'output':tf.Variable(tf.random_normal([n_hidden_2, n_output]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'output':tf.Variable(tf.random_normal([n_output]))
}

In [19]:
# Building the encoder
# def encoder(x):
#     # Encoder Hidden layer with sigmoid activation #1
#     layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
#                                    biases['encoder_b1']))
#     layer_1=tf.nn.dropout(layer_1, dropout_keep_prob) 
#     # Decoder Hidden layer with sigmoid activation #2
#     layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
#                                    biases['encoder_b2']))
#     return layer_2


# Building the decoder
def decoder(x):
    x = tf.reshape(x, shape=[-1,28,28, 1])
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['h1']),
                                   biases['b1']))
    layer_1=tf.nn.dropout(layer_1, dropout_keep_prob) 
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['h2']),
                                   biases['b2']))
    layer_2=tf.nn.dropout(layer_2, dropout_keep_prob) 
    
    output = tf.nn.sigmoid(tf.matmul(layer_2,weights['output']) + biases['output'])
    return output

# Construct model
# encoder_op = encoder(X)
decoder_op = decoder(x)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
# y_true = y

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_pred - y, 2))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [24]:
# Launch the graph
sess = tf.Session()

sess.run(init)

total_batch = int(mnist.train.num_examples/batch_size)
# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(num_batch): 
        randidx = np.random.randint(trainimg.shape[0], size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_xs_noisy = batch_xs + 0.3*np.random.randn(batch_xs.shape[0], 784)

        batch_ys = trainlabel[randidx, :]                

        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs_noisy, y: batch_xs, dropout_keep_prob: 0.5})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs_noisy, y: batch_xs, dropout_keep_prob: 1})/num_batch

    # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
    if epoch % plot_step == 0:
        # Test one
        randidx  = np.random.randint(testimg.shape[0], size=1)
        testvec  = testimg[randidx, :]
        noisyvec = testvec + 0.3*np.random.randn(1, 784)
        outvec   = sess.run(y_pred, feed_dict={x: testvec, dropout_keep_prob: 1.})
        outimg   = np.reshape(outvec, (28, 28))

        # Plot 
        plt.matshow(np.reshape(testvec, (28, 28)), cmap=plt.get_cmap('gray'))
        plt.title("[" + str(epoch) + "] Original Image")
        plt.colorbar()

        plt.matshow(np.reshape(noisyvec, (28, 28)), cmap=plt.get_cmap('gray'))
        plt.title("[" + str(epoch) + "] Input Image")
        plt.colorbar()

        plt.matshow(outimg, cmap=plt.get_cmap('gray'))
        plt.title("[" + str(epoch) + "] Reconstructed Image")
        plt.colorbar()
        plt.show()

#         saver.save(sess, savedir + 'dae_dr.ckpt', global_step=epoch)
print ("Optimization Finished!")

Epoch: 000/005 cost: 0.106128549
Epoch: 001/005 cost: 0.080328583
Epoch: 002/005 cost: 0.063283764
Epoch: 003/005 cost: 0.049055579
Epoch: 004/005 cost: 0.038002230
Optimization Finished!


In [ ]:
# Result